In [4]:
import json
import requests
import time

# Agencies
Total: 290

In [24]:
with open('./data/launch-library/agencies.json', encoding='utf-8') as f:
  agencies = json.load(f)
  print(len(agencies))

290


# Astronauts
Total: 721

In [16]:
with open('./data/launch-library/astronauts.json', encoding='utf-8') as f:
  astronauts = json.load(f)
  print(len(astronauts))

104


# Expeditions
Total: 147

In [26]:
with open('./data/launch-library/expeditions.json', encoding='utf-8') as f:
  expeditions = json.load(f)
  print(len(expeditions))

147


# Launches
Total: 6767

In [ ]:
max_offset = 68
sleep_time = 300

for i in range(10, max_offset):
	with open('./data/launch-library/launches.json', 'r+', encoding='utf-8') as f:
		launches = json.load(f)
		f.seek(0)
		data = requests.get('https://ll.thespacedevs.com/2.2.0/launch/?mode=detailed&limit=100&offset={}'.format(i * 100))
		for result in data.json()['results']:
			launches.append(result)
		json.dump(launches, f, indent=2)
		f.truncate()
		hours_left = (((max_offset - i) * sleep_time / 60) / 60)
		minutes_left = ((max_offset - i) * sleep_time / 60) % 60
		print('%i / 67 | %ih %2im remaining' % (i, hours_left, minutes_left))
		time.sleep(sleep_time)
print('done :)')


In [4]:

with open('./data/launch-library/launches.json', 'r+', encoding='utf-8') as f:
  launches = json.load(f)
  print(len(launches))

6767


# Locations
Total: 49

In [28]:
with open('./data/launch-library/locations.json', encoding='utf-8') as f:
  locations = json.load(f)
  print(len(locations))

49


# I want it all!
I want it now

In [ ]:
cheeky_export = {}

In [12]:
def get_all_remaining_endpoints():
  endpoints_info = {}
  sleep_seconds = 250

  with open('./data/launch-library/remaining_endpoints.json', 'r') as endpoints_json:
    remaining_endpoints = json.load(endpoints_json)
    
  for endpoint_name in remaining_endpoints:
    print('getting {}'.format(endpoint_name))
    endpoint_url = remaining_endpoints[endpoint_name]
    response = requests.get(endpoint_url, params={'mode':'detailed', 'limit':1}).json()
    if 'next' in response and 'count' in response:
      endpoints_info.update({endpoint_name: {'next': response['next'], 'count': response['count']}})
    else:
      print('[ERROR] getting {}\nreponse:'.format(endpoint_name))
      print(response)
    # with open('./data/launch-library/{}.json'.format(endpoint_name), 'w') as out_json:
    #   json.dump(response['results'], out_json, indent=2)
  
  cheeky_export = endpoints_info
  # cooldown_seconds = len(endpoints_info) * sleep_seconds
  # cooldown_minutes = (int)(cooldown_seconds / 60)
  # total_requests = sum([(int)(endpoints_info[ep]['count'] / 100) for ep in endpoints_info])
  total_requests = 30
  ###
  total_seconds = total_requests * sleep_seconds
  total_hours = (int)(total_seconds / 3600)
  total_minutes = (int)(total_seconds / 60) % 60
  print('Total requests needed: {}'.format(total_requests))
  print('This should take about {}h {}m'.format(total_hours, total_minutes))

  # print('sleeping for {}m'.format(cooldown_minutes))
  # time.sleep(cooldown_seconds)


  return

  for endpoint_name in endpoints_info:
    current_offset = 0
    while endpoints_info[endpoint_name]['next'] != None:
      current_offset += 1
      with open('./data/launch-library/{}.json'.format(endpoint_name), 'r+') as in_json:
        entries = json.load(in_json)
        in_json.seek(0)
        # next_response = requests.get(endpoints_info[endpoint_name]['next']).json()
        for result in next_response['results']:
          entries.append(result)
        json.dump(entries, in_json, indent=2)
        in_json.truncate()
        endpoints_info.update({endpoint_name: {'next': next_response['next'], 'count': next_response['count']}})

        endpoint_count_div_100 = (int)(endpoints_info[endpoint_name]['count'] / 100)
        seconds_remaining = endpoint_count_div_100 * sleep_seconds
        hours_remaining = (int)(seconds_remaining / 3600)
        minutes_remaining = (int)(seconds_remaining / 60) % 60
        print('%i / %i | %s | %ih %2im remaining' % (current_offset, endpoint_count_div_100, endpoint_name, hours_remaining, minutes_remaining))
        time.sleep(sleep_seconds)
    print('{} done :)'.format(endpoint_name))


In [18]:
cheeky_export

{}

In [13]:
get_all_remaining_endpoints()

getting astronauts
[ERROR] getting astronauts
reponse:
{'detail': 'Request was throttled. Expected available in 3116 seconds.'}
getting docking_events
[ERROR] getting docking_events
reponse:
{'detail': 'Request was throttled. Expected available in 3115 seconds.'}
getting launchers
[ERROR] getting launchers
reponse:
{'detail': 'Request was throttled. Expected available in 3115 seconds.'}
getting pads
[ERROR] getting pads
reponse:
{'detail': 'Request was throttled. Expected available in 3115 seconds.'}
getting spacestations
[ERROR] getting spacestations
reponse:
{'detail': 'Request was throttled. Expected available in 3114 seconds.'}
getting spacecraft_flights
[ERROR] getting spacecraft_flights
reponse:
{'detail': 'Request was throttled. Expected available in 3114 seconds.'}
getting spacecrafts
[ERROR] getting spacecrafts
reponse:
{'detail': 'Request was throttled. Expected available in 3114 seconds.'}
getting programs
[ERROR] getting programs
reponse:
{'detail': 'Request was throttled. 